In [58]:
import pandas as pd
import numpy as np
import math

from tqdm.notebook import tqdm
import time

In [2]:
path = 'C:/Users/Rusla/Documents/GitHub/Пушкинская карта - стажировка/Sample/'

In [3]:
users = pd.read_csv(path + 'users_full.csv', sep=';', error_bad_lines=False)

In [4]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879681 entries, 0 to 2879680
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   create_date         int64  
 1   user_id             object 
 2   user_birth          object 
 3   create_date.1       object 
 4   user_phone_details  object 
 5   buyer_mobile_phone  object 
 6   region              object 
 7   age                 int64  
 8   user_region         float64
dtypes: float64(1), int64(2), object(6)
memory usage: 197.7+ MB


In [5]:
users.sample(3)

,create_date,user_id,user_birth,create_date.1,user_phone_details,buyer_mobile_phone,region,age,user_region
1462406,1630588173,7e017fb6353666fa996a1c7704ebf0a2,2007-03-01,NaN,NaN,NaN,Курганская область,15,45.0
1040210,1630748255,5580b1fc82caff6e6eed2ef2ab3abb4e,2002-05-24,2021-11-30,"iOS, iOS 14.8.1",91730796578b6970a3fca467d9a6edfe,Москва,20,77.0
2334751,1637566017,a0805d5493e26ff143d878d3bb6d37cf,2007-08-31,2021-11-20,"iOS, iOS 15.0.1",50dbd8e8475188e04466a2ad0ece9798,NaN,14,NaN


In [6]:
users = users.astype({"age": "object", "user_region": "object"})

In [7]:
import json 
expl = [json.loads(line) for line in open(path + 'tickets2.json', 'r', encoding='utf-8', errors = 'ignore')]

In [8]:
tickets = pd.DataFrame(expl)

In [9]:
tickets.sample(3)

,payment_date,comment,session_event_id,session_date,session_params,owner,refund_date,buyer_mobile_phone,refund_reason,payment_amount,visitor_first_name,refund_ticket_price,create_date,status,payment_ticket_price,session_organization_id,visit_date
578780,1636484376,,1315538,1637758800,"Основной зал , Партер, 11 ряд, 22 место",7107520929,NaN,a12827c3a30c2a8cf4ebbdae5f238d3f,None,230.00,,None,1636484432114957838,visited,230.00,7765,1.637758e+09
393045,1638011101,,190992,1640779200,,5406014684,NaN,cabd06a7afd6a18c5c87bc7e2fa182be,None,3000,,None,1638011582478902809,visited,3000,5295,1.640776e+09
330198,1636998346,,1430860,1640102400,"сектор:Партер,ряд:6,место:21",7720312951,NaN,1c235f981c54ad1f8a5c6d448fb68a5c,None,1600,,None,1636998348515185707,visited,1600,28619,1.640102e+09


In [10]:
tickets[tickets['buyer_mobile_phone'].isna()]

,payment_date,comment,session_event_id,session_date,session_params,owner,refund_date,buyer_mobile_phone,refund_reason,payment_amount,visitor_first_name,refund_ticket_price,create_date,status,payment_ticket_price,session_organization_id,visit_date


In [11]:
tickets.visitor_first_name.nunique() # можно было попробовать пол по именм, но имён у нас всего 11 тысяч из 1,2 млн.

11791

# Добавим данные пользователя в дата сет с купленными билетами

In [12]:
tickets = pd.merge(tickets,users,on=['buyer_mobile_phone'], how='left')

In [13]:
tickets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 854330 entries, 0 to 854329
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   payment_date             854330 non-null  int64  
 1   comment                  854330 non-null  object 
 2   session_event_id         854330 non-null  object 
 3   session_date             854330 non-null  int64  
 4   session_params           854330 non-null  object 
 5   owner                    854330 non-null  object 
 6   refund_date              25117 non-null   float64
 7   buyer_mobile_phone       854330 non-null  object 
 8   refund_reason            25117 non-null   object 
 9   payment_amount           854330 non-null  object 
 10  visitor_first_name       854330 non-null  object 
 11  refund_ticket_price      24409 non-null   object 
 12  create_date_x            854330 non-null  int64  
 13  status                   854330 non-null  object 
 14  paym

In [14]:
tickets_2 = tickets[tickets['user_id'].notna() == True]

In [15]:
tickets_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530576 entries, 1 to 854329
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   payment_date             530576 non-null  int64  
 1   comment                  530576 non-null  object 
 2   session_event_id         530576 non-null  object 
 3   session_date             530576 non-null  int64  
 4   session_params           530576 non-null  object 
 5   owner                    530576 non-null  object 
 6   refund_date              16455 non-null   float64
 7   buyer_mobile_phone       530576 non-null  object 
 8   refund_reason            16455 non-null   object 
 9   payment_amount           530576 non-null  object 
 10  visitor_first_name       530576 non-null  object 
 11  refund_ticket_price      16025 non-null   object 
 12  create_date_x            530576 non-null  int64  
 13  status                   530576 non-null  object 
 14  paym

In [16]:
cliks_add = pd.read_csv(path + 'cliks_add.csv', sep=';', error_bad_lines=False)

In [17]:
cliks_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5747389 entries, 0 to 5747388
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   create_time              object 
 1   create_date              object 
 2   user_phone_details       object 
 3   buyer_mobile_phone       object 
 4   user_id                  object 
 5   url                      object 
 6   session_id               float64
 7   session_name             object 
 8   organization_id          float64
 9   organization_name        object 
 10  session_identity         object 
 11  Org_region_number        float64
 12  org_category             object 
 13  age                      float64
 14  user_region              float64
 15  user_phone_details_id    object 
 16  user_phone_details_id_2  int64  
dtypes: float64(5), int64(1), object(11)
memory usage: 745.4+ MB


In [18]:
cliks_add.shape

(5747389, 17)

In [19]:
ticket_users = tickets_2['user_id'].unique()

In [20]:
tickets_2['user_id'].nunique()

329522

In [21]:
len(ticket_users)

329522

In [22]:
cliks_add_users = cliks_add['user_id'].unique()

In [23]:
len(cliks_add_users)

272442

In [24]:
cliks_add['user_id'].nunique()

272442

In [25]:
cliks_add_users[:5]

array(['08014dcd786881f349938c9255e9df36',
       'b2560eaa8bc23dbba5f24aa5075b89ed',
       'fdfe16d86915392aeeca341505cce1c1',
       'cf3f95a25510513018a33990a30f5acb',
       'e2f896fb069e555a5faaf71a3d2b8dec'], dtype=object)

In [26]:
ticket_users[:5]

array(['ef54479d868c93e5b6a6e425fc780954',
       '68f0a5ca2209fedf6d574d74940e55a5',
       '89bc110254468dbf23f51f84385418a0',
       '38d83e61be8f2afd0d335686d685dc56',
       '65835d1c0954fe08db3ffa4e1a5fb75f'], dtype=object)

In [44]:
cliks_and_tickets = cliks_add.copy()

In [37]:
cliks_and_tickets.shape

(5747389, 17)

In [48]:
cliks_and_tickets = cliks_and_tickets.drop(np.where(cliks_and_tickets['user_id']
                                                    == 'b2560eaa8bc23dbba5f24aa5075b89ed')[1])

IndexError: tuple index out of range

In [46]:
cliks_and_tickets.shape

(5747369, 17)

In [50]:
cliks_and_tickets = pd.DataFrame()

%%time
users_t_and_c = []
for i in tqdm(cliks_add_users):
    if i in ticket_users:
        users_t_and_c.append(i)

In [62]:
%%time
for i in tqdm(cliks_add_users):
    if i in ticket_users:
        cliks_and_tickets = cliks_and_tickets.append([cliks_and_tickets, cliks_add[cliks_add['user_id'] == i]],
                                                    ignore_index = True, sort = False)

  0%|          | 0/272442 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [65]:
import time
start_time = time.time()

cliks_and_tickets = cliks_add.merge(
    tickets_2.drop_duplicates(subset=['user_id']), left_on='user_id', right_on=['user_id'], how='inner')

print("--- %s seconds ---" % (time.time() - start_time))

--- 15.785868644714355 seconds ---


In [66]:
cliks_and_tickets.shape

(3538545, 41)

In [68]:
import time
start_time = time.time()

cliks_and_tickets_2 = cliks_add.loc[cliks_add.user_id.isin(tickets_2.user_id.unique())] 

print("--- %s seconds ---" % (time.time() - start_time))

--- 1.6841983795166016 seconds ---


In [69]:
cliks_and_tickets_2.shape

(3538545, 17)

In [70]:
cliks_and_tickets_2.user_id.nunique()

137156

In [ ]:
cliks_and_tickets.to_csv(path + 'cliks_and_tickets.csv', sep = ';', )